In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import pylab as pl
#%matplotlib inline

qcd = pd.read_pickle('qcd.pkl')
fields  = qcd.keys().tolist()
nevents = len(qcd)
nevents, fields
def getParticles(qcd, evnumber, fields=['name', 
                                        'energy', 'px', 'py', 'pz', 
                                        'pt', 'y',  'phi', 'mt']):
    p  = qcd['Particle'][evnumber]
    
    # compute pT
    px = p['Particle_px']
    py = p['Particle_py']
    pt = np.sqrt(px**2 + py**2); p['pt'] = pt   
    
    # compute rapidity (y)
    pz = p['Particle_pz']
    # multiply by x > 1 to avoid divide by zero.
    E  = p['Particle_energy'] * 1.00000001 
    y  = np.log((E + pz)/(E - pz))/2; p['y']  = y

    # compute phi, transverse mass
    p['phi'] = np.arctan2(px, py)
    p['mt']  = np.sqrt(pt**2 + p['Particle_mass']**2)
    
    # shorten names
    p['name']   = p['Particle_name']
    p['energy'] = p['Particle_energy']
    p['px']     = p['Particle_px']
    p['py']     = p['Particle_py']
    p['pz']     = p['Particle_pz']
    
    # select final state partons with |y| < 5. Particles 
    # with |y| > 5 go down the LHC beam pipe.
    # d1 to d2 are the ordinal values of the daughters 
    # of a decayed particle. If d1 < 0, then the particle 
    # has no daughters. Final state particles are those 
    # without daughters.
    idx= (p['Particle_d1'] < 0) & (abs(p['y']) < 5)
    pp = p[idx]
    
    return pp[fields]
        
P=getParticles(qcd,0)
R=0.8
di=1/P['pt']**2
N=len(P)
indices = P.index
imin,jmin,dijmin=-1,-1,999999
for i in range(len(P)-1):
    Indexor=np.arange(i+1,len(P))
    for j in Indexor:
        DR2=(P.loc[indices[i]]['y']-P.loc[indices[j]]['y'])**2 + (P.loc[indices[i]]['phi']-P.loc[indices[j]]['phi'])**2
        DR2def=0.8**2
        dij=min(di.loc[indices[i]],di.loc[indices[j]])*(DR2/DR2def)
        print dij
        
        if dij<dijmin:
            dijmin=dij
            imin=indices[i]
            jmin=indices[j]

#print dijmin,imin,jmin
